# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,25.05,83,100,4.99,PN,1710405785
1,1,ushuaia,-54.8000,-68.3000,4.81,81,20,4.63,AR,1710405805
2,2,invercargill,-46.4000,168.3500,10.57,93,99,3.13,NZ,1710405805
3,3,aasiaat,68.7098,-52.8699,-19.42,99,100,1.39,GL,1710405806
4,4,bethel,41.3712,-73.4140,6.23,83,0,0.97,US,1710405806



### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
# Configure the map plot
cit_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 1000,
    frame_height = 800,
    size = "Humidity",
    scale = 1,
    color = "City",
    
)

# Display the map
cit_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrow_df = city_data_df[(city_data_df['Max Temp'] < 20 &(city_data_df['Max Temp'] > 22))]
                        


# Drop any rows with null values
city_data_narrow_df = city_data_narrow_df.dropna()

# Display sample data
city_data_narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,aasiaat,68.7098,-52.8699,-19.42,99,100,1.39,GL,1710405806
5,5,vorkuta,67.5000,64.0000,-15.17,93,94,3.45,RU,1710405807
7,7,udachny,66.4167,112.4000,-19.51,88,65,2.11,RU,1710405808
16,16,whitehorse,60.7161,-135.0538,-0.57,87,100,9.77,CA,1710405812
17,17,tiksi,71.6872,128.8694,-28.59,97,19,3.05,RU,1710405813
...,...,...,...,...,...,...,...,...,...,...
523,523,omutinskiy,56.4745,67.6709,-4.23,86,100,4.50,RU,1710406051
528,528,atbasar,51.8000,68.3333,-7.32,88,100,6.86,KZ,1710406053
544,544,ola,59.5833,151.2833,-6.75,76,100,4.03,RU,1710406058
552,552,narvik,68.4384,17.4272,-2.33,86,100,1.25,NO,1710406060


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_narrow_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,aasiaat,GL,68.7098,-52.8699,99,
5,vorkuta,RU,67.5000,64.0000,93,
7,udachny,RU,66.4167,112.4000,88,
16,whitehorse,CA,60.7161,-135.0538,87,
17,tiksi,RU,71.6872,128.8694,97,
...,...,...,...,...,...,...
523,omutinskiy,RU,56.4745,67.6709,86,
528,atbasar,KZ,51.8000,68.3333,88,
544,ola,RU,59.5833,151.2833,76,
552,narvik,NO,68.4384,17.4272,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "categories":"accommodation.hotel",
     "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aasiaat - nearest hotel: SØMA
vorkuta - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
tiksi - nearest hotel: Арктика
sernur - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
tulun - nearest hotel: Центральная
nemuro - nearest hotel: お宿エクハシ
yellowknife - nearest hotel: No hotel found
vuktyl - nearest hotel: Вуктыл
longyearbyen - nearest hotel: The Vault
iskateley - nearest hotel: No hotel found
college - nearest hotel: No hotel found
ridder - nearest hotel: No hotel found
sept-iles - nearest hotel: Château Arnaud
olonkinbyen - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
chengzihe - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
revda - nearest hotel: No hotel found
val-d'or - nearest hotel: No hotel found
nar'yan-mar - near

,City,Country,Lat,Lng,Humidity,Hotel Name
3,aasiaat,GL,68.7098,-52.8699,99,SØMA
5,vorkuta,RU,67.5000,64.0000,93,No hotel found
7,udachny,RU,66.4167,112.4000,88,No hotel found
16,whitehorse,CA,60.7161,-135.0538,87,Town & Mountain Hotel
17,tiksi,RU,71.6872,128.8694,97,Арктика
...,...,...,...,...,...,...
523,omutinskiy,RU,56.4745,67.6709,86,No hotel found
528,atbasar,KZ,51.8000,68.3333,88,Дана
544,ola,RU,59.5833,151.2833,76,No hotel found
552,narvik,NO,68.4384,17.4272,86,Narvik hotel Wivel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 1000,
    frame_height = 800,
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
hotel_plot 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)